<center>
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <h3><b>Objective</b></h3>
    <b>To delete unattached EBS volume using unSkript actions.</b>
</div>

<br>
</center>
<center><h2>Delete Unattached EBS Volume</h2></center>

# Steps Overview
   1)[Filter AWS Unattached EBS Volume](#1)</br>
   2)[Create Snapshot Of EBS Volume](#2)</br>
   3)[Delete EBS Volume](#3)

### <a id="1">Filter Unattached EBS Volumes</a>
Here we will use unSkript <b>Filter AWS Unattached EBS Volume</b> action. This action filters all the EBS volumes from the given region and returns a list of all the unattached EBS volumes.

>Input parameters: `region`

>Ouput variable: `unattached_volumes`

In [21]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint
from beartype import beartype


@beartype
def aws_filter_ebs_unattached_volumes_printer(output):
    if output is None:
        return
    pprint.pprint({"Volume IDs": output})


@beartype
def aws_filter_ebs_unattached_volumes(handle, region: str) -> List:
    """aws_filter_ebs_unattached_volumes Returns an array of ebs volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: Result of the API in the List form.
    """
    # Filtering the volume by region
    ec2Client = handle.resource('ec2',region_name=region)
    volumes = ec2Client.volumes.all()

    # collecting the volumes which has zero attachments
    result=[]
    for volume in volumes:
        if len(volume.attachments) == 0:
            result.append(volume.id)

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(outputName="unattached_volumes")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_ebs_unattached_volumes, lego_printer=aws_filter_ebs_unattached_volumes_printer, hdl=hdl, args=args)

### <a id="2">Create Snapshot Of EBS Volume</a>
Here we will use unSkript <b>Create Snapshot Of EBS Volume</b> action. In this action, we will back up the data stored in EBS volumes by passing the list of unattached EBS volumes from step 1 and creating a snapshot of the EBS volume of the EC2 instance.

>Input parameters: `volume_id`, `region`

>Ouput variable: `delete_status`

In [18]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint
from beartype import beartype


@beartype
def aws_create_volumes_snapshot_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_volumes_snapshot(handle, volume_id: str, region: str) -> List:
    """aws_create_volumes_snapshot Returns an list containing SnapshotId.

        :type region: string
        :param region: used to filter the volume for a given region.

        :type volume_id: string
        :param volume_id: Volume ID to create snapshot for given volume.

        :rtype: List containing SnapshotId.
    """
    result = []

    ec2Client = handle.resource('ec2', region_name=region)

    try:
        response = ec2Client.create_snapshot(VolumeId=volume_id)
        result.append(response)
    except Exception as e:
        result.append(e)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "region",
    "volume_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "unattached_volumes",
    "iter_parameter": "volume_id"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "create_snapshot == 'Yes' and len(unattached_volumes) > 0",
    "condition_result": true
    }''')
task.configure(outputName="snapshot_metadata")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_volumes_snapshot, lego_printer=aws_create_volumes_snapshot_printer, hdl=hdl, args=args)

### <a id="3">Delete EBS Volume</a>
Here we will use unSkript <b>Delete EBS Volume</b> action. In this action, we will pass the list of unattached EBS volumes from step 1 and delete those volumes.

>Input parameters: `volume_id`, `region`

>Ouput variable: `delete_status`

In [19]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint
from beartype import beartype


@beartype
def aws_delete_volumes_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_volumes(handle, volume_id: str, region: str) -> List:
    """aws_delete_volumes Returns array of delete volume status.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type region: string
        :param region: used to filter the volume for a given region.

        :type volume_id: string
        :param volume_id: Volume ID to delete a given volume.

        :rtype: Array of delete volume status.
    """
    result = []

    ec2Client = handle.client('ec2',region_name=region)

    # Adding logic for deletion criteria
    try:
        response = ec2Client.delete_volume(VolumeId=volume_id,)
        result.append(response)
    except Exception as e:
        result.append(e)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "region",
    "volume_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "unattached_volumes",
    "iter_parameter": "volume_id"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(unattached_volumes) > 0",
    "condition_result": true
    }''')
task.configure(outputName="delete_status")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_volumes, lego_printer=aws_delete_volumes_printer, hdl=hdl, args=args)

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS actions to filter unattached EBS volumes and delete those. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io